## LangSmith를 활용한 LLM Evaluation

### 문제 상황
- LLM 챗봇을 만들었는데 **실제로 잘 작동하는지** 모르겠다
- 프롬프트를 수정했는데 **성능이 나아졌는지 확인**하고 싶다
- 사용자에게 배포하기 전에 **품질을 객관적으로 검증**하고 싶다

### 평가가 필요한 이유
1. **객관적 성능 측정**: "느낌적으로 좋다" → "80% 정확도" 수치화
2. **버전 비교**: 프롬프트 v1 vs v2 중 뭐가 더 나은지 비교
3. **약점 발견**: 어떤 유형의 질문에서 자주 틀리는지 파악
4. **지속적 개선**: 평가 → 개선 → 재평가 사이클 구축
5. **신뢰도 확보**: 특히 법률, 의료, 금융 같은 고위험 도메인

### LangSmith를 활용한 답변 평가
- LangSmith는 **LLM 애플리케이션의 평가 및 모니터링 도구**
- LangSmith의 역할
    - 평가 자동 반복 실행
    - 결과 저장 및 DB 관리
    - 웹 UI로 결과 시각화
    - 버전별 성능 비교
    - 팀 협업 기능

## LLM 답변 평가 전체 프로세스 
- 1. 평가 데이터셋 구축 (채점지) 
- 2. 평가 대상 (Target) 함수 정의
- 3. 평가자 (Evaluator) 정의
- 4. LangSmith로 평가 실행: evaluate() 
- 5. LangSmith 웹 UI에서 결과 확인

### 1. 평가 데이터셋 구축 (채점지) 
- Evaluation에 활용될 Question - Answer pair 생성

In [ ]:
from langsmith import Client

client = Client()

# Define dataset: these are your test cases
dataset_name = "income_tax_dataset"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    inputs=[
        {"input_question": "제1조에 따른 소득세법의 목적은 무엇인가요?"},
        {"input_question": "'거주자'는 소득세법에서 어떻게 정의되나요?"},
        {"input_question": "'비거주자'는 소득세법에 따라 어떻게 정의되나요?"},
        {"input_question": "소득세법에 따른 '내국법인'은 누구를 의미하나요?"},
        {"input_question": "소득세법에 따라 소득세를 납부할 의무가 있는 사람은 누구인가요?"},
        {"input_question": "거주자의 과세 범위는 무엇인가요?"},
        {"input_question": "소득세법에 따라 소득은 어떻게 분류되나요?"},
        {"input_question": "종합소득이란 무엇인가요?"},
        {"input_question": "세금이 면제되는 소득의 종류는 무엇인가요?"},
        {"input_question": "소득세의 과세기간은 어떻게 되나요?"},
        {"input_question": "거주자의 소득세 납세지는 어디인가요?"},
        {"input_question": "비거주자의 소득세 납세지는 어디인가요?"},
        {"input_question": "납세지가 불분명한 경우 어떻게 되나요?"},
        {"input_question": "원천징수세액의 납세지는 어떻게 결정되나요?"},
        {"input_question": "납세자의 사망 시 납세지는 어떻게 되나요?"},
        {"input_question": "신탁 소득에 대한 납세의 범위는 무엇인가요?"},
        {"input_question": "원천징수 대상 소득은 무엇인가요?"},
        {"input_question": "공동 소유 자산의 양도소득은 어떻게 과세되나요?"},
        {"input_question": "이자 소득의 출처는 무엇인가요?"},
        {"input_question": "소득세법에서 배당소득은 어떻게 정의되나요?"}
    ],
    outputs=[
        {"output_answer": "소득세법의 목적은 소득의 성격과 납세자의 부담능력에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지하는 것입니다."},
        {"output_answer": "'거주자'는 한국에 주소를 두거나 183일 이상 거소를 둔 개인을 의미합니다."},
        {"output_answer": "'비거주자'는 거주자가 아닌 개인을 의미합니다."},
        {"output_answer": "'내국법인'은 법인세법 제2조 제1호에 따른 내국법인을 의미합니다."},
        {"output_answer": "거주자 및 국내원천소득이 있는 비거주자는 소득세를 납부할 의무가 있습니다."},
        {"output_answer": "거주자는 법에서 규정한 모든 소득에 대해 과세되며, 비거주자는 국내원천소득에 대해서만 과세됩니다."},
        {"output_answer": "소득은 종합소득, 퇴직소득, 양도소득으로 분류됩니다."},
        {"output_answer": "종합소득은 이자소득, 배당소득, 사업소득, 근로소득, 연금소득 및 기타소득을 포함합니다."},
        {"output_answer": "비과세 소득에는 공익신탁의 이익, 특정 사업소득 및 기타 법에서 정한 특정 소득이 포함됩니다."},
        {"output_answer": "소득세의 과세기간은 매년 1월 1일부터 12월 31일까지입니다."},
        {"output_answer": "거주자의 소득세 납세지는 주소지이며, 주소지가 없으면 거소지입니다."},
        {"output_answer": "비거주자의 소득세 납세지는 국내사업장의 소재지입니다. 국내사업장이 여러 곳인 경우 주된 사업장의 소재지가 납세지가 됩니다."},
        {"output_answer": "납세지가 불분명한 경우 대통령령으로 정합니다."},
        {"output_answer": "원천징수세액의 납세지는 원천징수자의 종류와 위치에 따라 결정됩니다."},
        {"output_answer": "납세자의 사망 시 상속인 또는 납세관리인의 주소지나 거소지가 납세지가 됩니다."},
        {"output_answer": "신탁 소득에 대한 납세의 범위는 신탁의 수익자가 해당 소득에 대해 납세의무를 집니다."},
        {"output_answer": "이자소득, 배당소득 및 기타 법에서 정한 소득은 원천징수 대상입니다."},
        {"output_answer": "공동 소유 자산의 양도소득은 각 거주자 소유 지분에 따라 과세됩니다."},
        {"output_answer": "이자 소득의 출처는 정부 및 지방자치단체가 발행한 채권, 법인이 발행한 채권, 국내외 은행 예금 등입니다."},
        {"output_answer": "배당소득은 국내외 법인으로부터 받는 배당금 및 배분금, 기타 법에서 정한 소득을 포함합니다."}
    ],
    metadata= [
        {"contexts": "제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다."},
        {"contexts": "제1조의2(정의) “거주자”란 국내에 주소를 두거나 183일 이상의 거소를 둔 개인을 말한다."},
        {"contexts": "제1조의2(정의) “비거주자”란 거주자가 아닌 개인을 말한다."},
        {"contexts": "제1조의2(정의) “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다."},
        {"contexts": "제2조(납세의무) 거주자 및 국내원천소득이 있는 비거주자는 소득세를 납부할 의무가 있다."},
        {"contexts": "제3조(과세소득의 범위) 거주자는 법에서 규정한 모든 소득에 대해 과세되며, 비거주자는 국내원천소득에 대해서만 과세된다."},
        {"contexts": "제4조(소득의 구분) 소득은 종합소득, 퇴직소득, 양도소득으로 분류된다."},
        {"contexts": "제4조(소득의 구분) 종합소득은 이자소득, 배당소득, 사업소득, 근로소득, 연금소득 및 기타소득을 포함한다."},
        {"contexts": "제12조(비과세소득) 비과세 소득에는 공익신탁의 이익, 특정 사업소득 및 기타 법에서 정한 특정 소득이 포함된다."},
        {"contexts": "제5조(과세기간) 소득세의 과세기간은 매년 1월 1일부터 12월 31일까지이다."},
        {"contexts": "제6조(납세지) 거주자의 소득세 납세지는 주소지이며, 주소지가 없으면 거소지이다."},
        {"contexts": "제6조(납세지) 비거주자의 소득세 납세지는 국내사업장의 소재지이다. 국내사업장이 여러 곳인 경우 주된 사업장의 소재지이다."},
        {"contexts": "제6조(납세지) 납세지가 불분명한 경우에는 대통령령으로 정한다."},
        {"contexts": "제7조(원천징수 등의 경우의 납세지) 원천징수세액의 납세지는 원천징수자의 종류와 위치에 따라 결정된다."},
        {"contexts": "제8조(상속 등의 경우의 납세지) 납세자의 사망 시 상속인 또는 납세관리인의 주소지나 거소지가 납세지가 된다."},
        {"contexts": "제2조의3(신탁재산 귀속 소득에 대한 납세의무의 범위) 신탁 소득에 대한 납세의 범위는 신탁의 수익자가 해당 소득에 대해 납세의무를 진다."},
        {"contexts": "제14조(과세표준의 계산) 이자소득, 배당소득 및 기타 법에서 정한 소득은 원천징수 대상이다."},
        {"contexts": "제14조(과세표준의 계산) 공동 소유 자산의 양도소득은 각 거주자 소유 지분에 따라 과세된다."},
        {"contexts": "제16조(이자소득) 이자 소득의 출처는 정부 및 지방자치단체가 발행한 채권, 법인이 발행한 채권, 국내외 은행 예금 등이다."},
        {"contexts": "제17조(배당소득) 배당소득은 국내외 법인으로부터 받는 배당금 및 배분금, 기타 법에서 정한 소득을 포함한다."}
    ],
    dataset_id=dataset.id,
)

### 2. 평가 대상 (Target) 함수 정의

In [ ]:
# 테스트: 함수가 제대로 작동하는지 확인
test_example = {"input_question": "거주자는 누구인가요?"}
result = predict_rag_answer(test_example)
print(f"질문: {test_example['input_question']}")
print(f"답변: {result['answer']}")

In [ ]:
# streamlit_chatbot 모듈을 import하기 위한 경로 추가
import sys
sys.path.append('../streamlit_chatbot')

# 실제 프로덕션 챗봇 로직을 import
from llm import get_ai_response_for_eval

In [ ]:
def predict_rag_answer(example: dict):
    """
    평가 대상 함수 (Target Function)
    - 데이터셋의 각 질문에 대해 챗봇이 답변을 생성
    - 실제 프로덕션 로직을 그대로 사용 (keyword dictionary, few-shot, RAG 모두 포함)
    
    Args:
        example: 데이터셋의 한 예제 {"input_question": "질문..."}
    
    Returns:
        {"answer": "챗봇의 답변..."}
    """
    question = example["input_question"]
    answer = get_ai_response_for_eval(question)
    return {"answer": answer}

### ✅ 2단계 완료!

**중요 포인트:**
- ✅ 실제 프로덕션 로직을 그대로 재사용
- ✅ Keyword Dictionary 체인 적용됨
- ✅ Few-shot 예시 적용됨
- ✅ RAG (Retriever + 프롬프트) 모두 적용됨
- ⚠️ History만 제외 (evaluation은 독립적인 질문이므로)

**다음 단계:**
- 3단계: Evaluator (평가자) 정의
- 4단계: evaluate() 실행
- 5단계: 결과 확인